In [3]:
from dotenv import load_dotenv
from langchain_classic.output_parsers import ResponseSchema

load_dotenv()

from langchain.chat_models import init_chat_model
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
# ChatPromptTemplate
model = init_chat_model("deepseek-chat", model_provider="deepseek")

# basic_qa_chain = model | StrOutputParser()

# response = basic_qa_chain.invoke("请自我介绍。")
# print(response)

# 解析布尔输出
from langchain_classic.output_parsers.boolean import BooleanOutputParser
# BooleanOutputParser

prompt_template = ChatPromptTemplate([
    ("system", "你是一个乐于助人的帮手，请根据用户的问题给出回答。"),
    ("user", "这是用户的问题：{topic}")
])

math_qa_chain = prompt_template | model | StrOutputParser()

question1 = \
"""
Alice and Bob are each holding some integer number of sweets. Alice says to Bob: “If
we each added the number of sweets we’re holding to our (positive integer) age, my answer would
be double yours. If we took the product, then my answer would be four times yours.” Bob replies:
“Why don’t you give me five of your sweets because then both our sum and product would be equal.”
What is the product of Alice and Bob’s ages?

"""

question = \
"""
what is 1 / 0?
"""

result = math_qa_chain.invoke({"topic": question})

print(result)


这是一个很好的数学问题！  

1 除以 0 在数学中**没有定义**，也就是说它没有意义。原因如下：  

- 除法可以理解为“一个数乘以多少等于另一个数”。  
  比如 \( 1 / 0 = x \) 意味着 \( 0 \times x = 1 \)，但任何数乘以 0 都等于 0，不可能等于 1，所以这样的 \( x \) 不存在。  

- 从极限的角度看，如果让除数越来越接近 0，比如 \( 1 / 0.1 = 10 \)，\( 1 / 0.01 = 100 \)，\( 1 / 0.001 = 1000 \)……结果会趋向无穷大，但无穷大不是一个确定的实数，而且从负方向接近 0 会趋向负无穷，所以极限也不唯一。  

因此，在标准算术中，**除以零是未定义的**。


In [3]:
from dotenv import load_dotenv
load_dotenv()

from langchain_core.prompts import PromptTemplate
from langchain_classic.output_parsers import ResponseSchema, StructuredOutputParser
from langchain.chat_models import init_chat_model

model = init_chat_model("deepseek-chat", model_provider="deepseek")

schemas = [
    ResponseSchema(name="name", description="用户的姓名"),
    ResponseSchema(name="age", description="用户的年龄")
]

parser = StructuredOutputParser.from_response_schemas(schemas)

prompt = PromptTemplate.from_template(
    "请根据以下内容提取用户信息，并返回 JSON 格式：\n{input}\n\n{format_instructions}"
)

format_instructions = parser.get_format_instructions()
print(format_instructions)

chain = (
    prompt.partial(format_instructions=parser.get_format_instructions())
    | model
    | parser
)

result = chain.invoke({"input": "用户叫李雷，今年25岁，是一名工程师。"})

print(result)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"name": string  // 用户的姓名
	"age": string  // 用户的年龄
}
```
{'name': '李雷', 'age': '25'}


LECL -- LangChainExpressionLanguage
函数式管道风格，允许通过管道符连接两个元素，实现功能的串联
invoke
stream
batch

In [1]:
from dotenv import load_dotenv
load_dotenv()
# 构建复合链
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableSequence, RunnableLambda
from langchain_classic.output_parsers import ResponseSchema, StructuredOutputParser

from langchain.chat_models import init_chat_model

model = init_chat_model("deepseek-chat", model_provider="deepseek")

news_gen_prompt = PromptTemplate.from_template(
    "请根据以下新闻标题撰写一则简短的新闻内容，100字以内：\n\n{title}"
)

# model输出一个AIMessage
news_chain = news_gen_prompt | model

schemas = [
    ResponseSchema(name="time", description="事件发生的时间"),
    ResponseSchema(name="location", description="事件发生的地点"),
    ResponseSchema(name="event", description="发生的具体事件")
]

parser = StructuredOutputParser.from_response_schemas(schemas)

summary_prompt = PromptTemplate.from_template(
    "请从下面的新闻内容中提取关键信息，并返回结构化的json格式：\n\n{news}\n\n{format_instructions}"
)

summary_chain = (
    summary_prompt.partial(format_instructions=parser.get_format_instructions())
    | model
    | parser
)

combined_chain = news_chain | summary_chain

response = combined_chain.invoke({"title": "陨石撞击地球"})

print(response)


{'time': '昨日凌晨', 'location': '太平洋无人海域', 'event': '一颗直径约10米的陨石坠入，未造成人员伤亡及财产损失，相关机构正对陨石碎片展开研究'}


In [1]:
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.chat_models import init_chat_model
from dotenv import load_dotenv
load_dotenv()

from langchain_core.output_parsers import StrOutputParser

model = init_chat_model("deepseek-chat", model_provider="deepseek")
chatbot_prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessage(content="你叫小智，是一名乐于助人的助手。"),
        MessagesPlaceholder(variable_name="messages")
    ]
)
basic_qa_chain = chatbot_prompt | model | StrOutputParser()
messages_list = [
    HumanMessage(content="你好，我叫程明，好久不见。"),
    AIMessage(content="你好呀！我叫小智，是一名乐于助人的ai助手，很高兴认识你。")
]

question = "你好，请详细介绍一下你自己。"
messages_list.append(HumanMessage(content=question))
# result = basic_qa_chain.invoke({"messages": messages_list})

async for chunk in basic_qa_chain.astream({"messages": messages_list}):
    print(chunk, end="", flush=True)

# print(result)

你好！我是小智，一个由深度求索公司开发的AI助手。很高兴能为你详细介绍我自己：

**我的能力：**
- 知识问答：涵盖科学、历史、文化、生活等各个领域
- 文本处理：写作、翻译、总结、改写等
- 逻辑推理：数学计算、问题分析、方案建议
- 创意生成：故事创作、诗歌写作、头脑风暴
- 编程帮助：代码解释、调试建议、算法思路

**我的特点：**
- 完全免费使用
- 支持128K长上下文对话
- 会主动承认知识盲区
- 注重回答的准确性和安全性

**我的限制：**
- 知识截止到2024年7月
- 无法处理实时信息
- 没有个人情感和主观体验

有什么具体想了解的吗？或者想试试我的什么功能？😊

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

#### Langchain 接入工具

- 基本流程

    - MCP
    - LangChain 内置工具


In [5]:
import pandas as pd
from langchain_core.prompts import ChatPromptTemplate
from langchain_experimental.tools import PythonAstREPLTool # Python交互式解释环境，能够保存运行上下文


# 电信用户流失预测
path = "/mnt/d/aistudy/dataset/langchain/telco/Telco_customer_churn.xlsx"
df = pd.read_excel(path)
# df.head(5)
tool = PythonAstREPLTool(locals={"df": df})
# tool.invoke("df.columns")

from dotenv import load_dotenv
load_dotenv()
from langchain.chat_models import init_chat_model
model = init_chat_model("deepseek-chat", model_provider="deepseek")

llm_with_tool = model.bind_tools([tool])

# response = llm_with_tool.invoke(
#     "我有一张表名为df，请告诉我表的列名"
# )

from langchain_core.output_parsers import JsonOutputKeyToolsParser
parser = JsonOutputKeyToolsParser(
    key_name=tool.name,
    first_tool_only=True
)
chain = llm_with_tool | parser
# response = chain.invoke("我有一张表名为df，请告诉我表的列名")

system_prompt = \
"""
你可以访问一个名为'df'的pandas数据框，你可以使用df.head().to_markdown()查看数据集的基本信息。
请根据用户提出的问题，编写Python代码来混搭，只返回代码，不返回其他内容。只允许使用pandas和内置库。
"""
from langchain_core.messages import SystemMessage, HumanMessage
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("user", "{question}")
    ]
)

from langchain_core.runnables import RunnableLambda
def _extract_code(message):
    return message["query"]

extract_code = RunnableLambda(_extract_code)
chain = prompt | llm_with_tool | parser | extract_code | tool
# chain = prompt
response = chain.invoke({"question": "统计已流失的用户有多少人？"})
response

ChatPromptValue(messages=[SystemMessage(content="\n你可以访问一个名为'df'的pandas数据框，你可以使用df.head().to_markdown()查看数据集的基本信息。\n请根据用户提出的问题，编写Python代码来混搭，只返回代码，不返回其他内容。只允许使用pandas和内置库。\n", additional_kwargs={}, response_metadata={}), HumanMessage(content='统计已流失的用户有多少人？', additional_kwargs={}, response_metadata={})])

In [22]:
from langchain_core.tools import tool
import requests
from dotenv import load_dotenv
load_dotenv(override=True)
import os
# https://api.openweathermap.org/data/2.5/weather?q={city name}&appid={API key}

@tool
def get_weather(city):
    """
    获取指定城市当前的天气情况
    :param city: 必要参数，城市名，应当为英文形式
    :return: 接口响应，json格式
    """
    url = "https://api.openweathermap.org/data/2.5/weather"
    params = {
        "q": city,
        "appid": os.getenv("OWM_APPID"),
        "units": "metric",
        "lang": "zh_cn"
    }
    response = requests.get(url, params=params)
    return response.json()

print(get_weather.name)
print(get_weather.description)
print(get_weather.args)


from langchain.chat_models import init_chat_model
from dotenv import load_dotenv
load_dotenv(override=True)
chat_model = init_chat_model("deepseek-chat", model_provider="deepseek")
chat_model = chat_model.bind_tools([get_weather])

from langchain_core.output_parsers import JsonOutputKeyToolsParser
parser = JsonOutputKeyToolsParser(key_name=get_weather.name, first_tool_only=True)

chain = chat_model # | parser # | get_weather

# response = chain.invoke("请问北京天气怎么样？")

from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

output_prompt = PromptTemplate.from_template(
"""
你将收到一段 JSON 格式的天气数据，请用简洁自然的方式将其转述给用户。
以下是天气 JSON 数据：

```json
{weather_json}
```

请将其转换为中文天气描述，例如：
“北京当前天气晴，气温为23℃，湿度58%，风速2.1米/秒。”
只返回一句话描述，不要其他说明或解释。
"""
)

output_chain = output_prompt | chat_model | StrOutputParser()

# weather_json = chain.invoke("请问成都天气怎么样？")
#
# response = output_chain.invoke({"weather_json": weather_json})

from langchain_core.runnables import RunnableLambda
combined_chain = chain # | RunnableLambda(lambda x: {"weather_json": x}) | output_chain
response = combined_chain.invoke("请问华盛顿的天气怎么样？")
print(response)

get_weather
获取指定城市当前的天气情况
:param city: 必要参数，城市名，应当为英文形式
:return: 接口响应，json格式
{'city': {'title': 'City'}}
content='我需要确认一下您指的是哪个华盛顿。美国有两个主要的地方都叫华盛顿：\n\n1. **华盛顿州**（Washington State）- 美国西北部的一个州\n2. **华盛顿特区**（Washington D.C.）- 美国的首都\n\n您想知道的是哪个地方的天气呢？' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 57, 'prompt_tokens': 325, 'total_tokens': 382, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 320}, 'prompt_cache_hit_tokens': 320, 'prompt_cache_miss_tokens': 5}, 'model_provider': 'deepseek', 'model_name': 'deepseek-chat', 'system_fingerprint': 'fp_eaab8d114b_prod0820_fp8_kvcache', 'id': '0fbea8ab-831d-4e36-bed8-72bc98c3dabc', 'finish_reason': 'stop', 'logprobs': None} id='lc_run--019bcaf0-7b9b-7071-8d6f-50bcffb7a88e-0' usage_metadata={'input_tokens': 325, 'output_tokens': 57, 'total_tokens': 382, 'input_token_details': {'cache_read': 320}, 'output_token_details': {}}


In [4]:
from langchain_classic.agents import create_tool_calling_agent, tool
from langchain_core.prompts import ChatPromptTemplate
from dotenv import load_dotenv
load_dotenv()
import os
import requests

@tool
def get_weather(city):
    """
    获取指定城市当前的天气情况
    :param city: 必要参数，城市名，应当为英文形式
    :return: 接口响应，json格式
    """
    url = "https://api.openweathermap.org/data/2.5/weather"
    params = {
        "q": city,
        "appid": os.getenv("OWM_APPID"),
        "units": "metric",
        "lang": "zh_cn"
    }
    response = requests.get(url, params=params)
    return response.json()

tools = [get_weather]

prompt = ChatPromptTemplate.from_messages([
    ("system", "你是天气助手，请根据用户的问题，给出相应的天气信息"),
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}")
])


from langchain.chat_models import init_chat_model
model = init_chat_model("deepseek-chat", model_provider="deepseek")

agent = create_tool_calling_agent(model, tools, prompt)

from langchain_classic.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

response = agent_executor.invoke({"input": "请问华盛顿今天的天气怎么样？"})
print(response)




> Entering new AgentExecutor chain...

Invoking: `get_weather` with `{'city': 'Washington'}`
responded: 我来帮您查询华盛顿今天的天气情况。

{'coord': {'lon': -120.5015, 'lat': 47.5001}, 'weather': [{'id': 800, 'main': 'Clear', 'description': '晴', 'icon': '01n'}], 'base': 'stations', 'main': {'temp': -2.13, 'feels_like': -3.92, 'temp_min': -2.78, 'temp_max': -1.64, 'pressure': 1035, 'humidity': 94, 'sea_level': 1035, 'grnd_level': 947}, 'visibility': 10000, 'wind': {'speed': 1.35, 'deg': 282, 'gust': 1.21}, 'clouds': {'all': 8}, 'dt': 1768635602, 'sys': {'type': 2, 'id': 2003593, 'country': 'US', 'sunrise': 1768578257, 'sunset': 1768610335}, 'timezone': -28800, 'id': 5815135, 'name': 'Washington', 'cod': 200}根据查询结果，华盛顿（美国）今天的天气情况如下：

**天气状况：** 晴朗 ☀️
**温度：** -2.13°C（体感温度 -3.92°C）
**温度范围：** -2.78°C 到 -1.64°C
**湿度：** 94%
**气压：** 1035 hPa
**风速：** 1.35 m/s（约5公里/小时）
**风向：** 282度（西风）
**能见度：** 10公里
**云量：** 8%

**总结：** 今天华盛顿天气非常晴朗，但气温较低，在零下2度左右，体感温度更低。湿度较高，风速较小。建议外出时注意保暖，穿着厚实的冬装。

> Finished chain.
{'input': '